In [ ]:
from ppr.ppr import PPR
from psiqworkbench import QPU, Qubits, Units
from random import randint

num_qubits = 4
# num_qubits = randint(1, 4)
x_mask = randint(0, 2**num_qubits - 1)
z_mask = randint(0, 2**num_qubits - 1)
theta = float(randint(0, 360))  # degrees

qpu = QPU(num_qubits=num_qubits, filters=[">>unitary>>",">>buffer>>"])
qubits = Qubits(qpu=qpu, num_qubits=num_qubits)

pq_qpu = QPU(num_qubits=num_qubits, filters=[">>unitary>>",">>buffer>>"])
pq_qubits = Qubits(qpu=pq_qpu, num_qubits=num_qubits)

print(f"num_qubits={num_qubits}, x_mask={x_mask:0{num_qubits}b}, z_mask={z_mask:0{num_qubits}b}, theta={theta} degrees")

In [ ]:
# from qubitization import AsymmetricQubitization, OracleA, OracleB
import importlib
aq_module = importlib.import_module("syk-simulation.qubitization.asymmetric_qubitization")
from psiqworkbench import QPU, Qubits
from workbench_algorithms.utils.paulimask import PauliMask
import numpy as np

def generate_pauli_strings(num_terms, num_system_qubits):
    pauli_strings = []
    paulis = ['I', 'X', 'Y', 'Z']
    for _ in range(num_terms):
        term = ''.join(np.random.choice(paulis, size=num_system_qubits))
        pauli_strings.append(" ".join(f"{p}{i}" for i, p in enumerate(term)))
    return pauli_strings    
# 8, 70, 5)
N = 8  # number of Majorana fermions, 100 is interesting
num_terms = 70 # number of Pauli terms in the Hamiltonian
depth = 5 # depth of random circuit used in asymmetric PREPARE


num_system_qubits = int(np.ceil(N/2))
num_index_qubits = int(np.ceil(np.log2(num_terms)))

total_qubits = int(num_system_qubits + num_index_qubits + 1 )
qpu = QPU(num_qubits=total_qubits, filters=[">>buffer>>"])

branch = Qubits(1, "branch", qpu=qpu)
index = Qubits(num_index_qubits, "index", qpu=qpu)
system = Qubits(num_system_qubits, "system", qpu=qpu)

AQ = aq_module.AsymmetricQubitization()

pauliStrings = generate_pauli_strings(num_terms, num_system_qubits)

AQ.compute(branch=branch, index=index, system=system, depth=depth, terms=pauliStrings, debug=True)


In [ ]:
qpu.draw(show_qubricks=True)

In [ ]:
print(qpu.witness)

In [ ]:
qpu.serialize("syk_aq_N8.basq.dataset.json", dialect="basquiat")

In [ ]:
for idx, pauli_string in enumerate(pauliStrings):
    x_mask, z_mask = PauliMask.from_pauli_string(" ".join( f"{p}{i}" for i, p in enumerate(pauli_string))).mask
    system.x(x_mask, cond=index == idx)
    system.z(z_mask, cond=index == idx)

In [ ]:
branch.had()
oracleA.compute(index=index, depth=depth, ctrl=branch == 0)
oracleB.compute(index=index, ctrl=branch == 1)

for idx, pauli_string in enumerate(pauliStrings):
    x_mask, z_mask = PauliMask.from_pauli_string(" ".join( f"{p}{i}" for i, p in enumerate(pauli_string))).mask
    system.x(x_mask, cond=index == idx)
    system.z(z_mask, cond=index == idx)

oracleA.uncompute()
oracleB.uncompute()

branch.x()
index.x()
index.z(cond=branch == 1)
branch.x()
index.x()

In [ ]:
qpu.draw()

In [ ]:
AQ.compute()
_compute(self, branch: Qubits, index: Qubits, system: Qubits, depth: int = 5, terms: list[str] = None):

In [ ]:
from psiqworkbench import QPU, Qubits
num_qubits = 4
x_mask = 0b0000
z_mask = 0b0000
theta = 90.0

qpu = QPU(num_qubits=num_qubits, filters=[">>unitary>>",">>buffer>>"])
qubits = Qubits(qpu=qpu, num_qubits=num_qubits)

qubits.ppr(theta=theta, x_mask=x_mask, z_mask=z_mask)
ufilter = qpu.get_filter_by_name('>>unitary>>')
matrix = ufilter.get()
print(f"x_mask={x_mask:0{num_qubits}b}, z_mask={z_mask:0{num_qubits}b}, theta={theta} degrees")
print(matrix)
qpu.nop(repeat=20)
qpu.draw()

In [ ]:
test_num_qubits = 4
test_x_mask = 0b0000
test_z_mask = 0b0000
test_theta = 90.0

# test_x_mask = randint(0, 2**test_num_qubits - 1)
# test_z_mask = randint(0, 2**test_num_qubits - 1)
# test_theta = float(randint(0, 360))  # degrees

print(f"num_qubits={test_num_qubits}, x_mask={test_x_mask:0{test_num_qubits}b}, z_mask={test_z_mask:0{test_num_qubits}b}, theta={test_theta} degrees")

In [ ]:
test_qpu = QPU(num_qubits=test_num_qubits, filters=[">>unitary>>",">>buffer>>"])
test_qubits = Qubits(qpu=test_qpu, num_qubits=test_num_qubits)

test_ppr = PPR()
test_ppr.compute(test_qubits, test_theta, test_x_mask, test_z_mask)
# print(f"x_mask={test_x_mask:0{test_num_qubits}b}, z_mask={test_z_mask:0{test_num_qubits}b}, theta={test_theta} degrees")
# test_qpu.nop(repeat=5)
# test_qpu.draw()
ufilter = test_qpu.get_filter_by_name('>>unitary>>')
matrix = ufilter.get()
print(matrix)

In [ ]:
pq_qpu = QPU(num_qubits=test_num_qubits, filters=[">>unitary>>",">>buffer>>"])
pq_qubits = Qubits(qpu=pq_qpu, num_qubits=test_num_qubits)
pq_qubits.ppr(theta=test_theta, x_mask=test_x_mask, z_mask=test_z_mask)
pq_ufilter = pq_qpu.get_filter_by_name('>>unitary>>')
pq_matrix = pq_ufilter.get()
print(pq_matrix)
pq_qpu.nop(repeat=15)
# print(f"x_mask={test_x_mask:0{test_num_qubits}b}, z_mask={test_z_mask:0{test_num_qubits}b}, theta={test_theta} degrees")
pq_qpu.draw()
# test_qpu.draw()

In [ ]:
ppr = PPR()
ppr.compute(qubits, theta=theta, x_mask=x_mask, z_mask=z_mask)

ufilter = qpu.get_filter_by_name('>>unitary>>')
matrix = ufilter.get()
matrix

In [ ]:
qpu.draw()

In [ ]:
pq_qpu.reset(num_qubits)
pq_qubits.ppr(theta=theta, x_mask=x_mask, z_mask=z_mask)

pq_ufilter = pq_qpu.get_filter_by_name('>>unitary>>')
pq_matrix = pq_ufilter.get()
pq_matrix

In [ ]:
pq_qpu.draw()

In [ ]:
from psiqworkbench import QPU, Qubits
num_qubits = 4
x_mask = 0b0000
z_mask = 0b0000

theta = 90.0
theta2 = 45.0

qpu = QPU(num_qubits=num_qubits, filters=[">>unitary>>",">>buffer>>"])
qubits = Qubits(qpu=qpu, num_qubits=num_qubits)

qubits.ppr(theta=theta, x_mask=x_mask, z_mask=z_mask)
ufilter = qpu.get_filter_by_name('>>unitary>>')
matrix = ufilter.get()
print(f"x_mask={x_mask:0{num_qubits}b}, z_mask={z_mask:0{num_qubits}b}, theta={theta} degrees")
print(matrix)
qpu.nop(repeat=20)
qpu.draw()
qpu.reset(num_qubits)
qubits.ppr(theta=theta2, x_mask=x_mask, z_mask=z_mask)
ufilter = qpu.get_filter_by_name('>>unitary>>')
matrix = ufilter.get()
print(f"x_mask={x_mask:0{num_qubits}b}, z_mask={z_mask:0{num_qubits}b}, theta={theta} degrees")
print(matrix)
qpu.nop(repeat=20)
qpu.draw()

In [ ]:
from psiqworkbench import QPU, Qubits
from psiqworkbench.qubricks import Reflect

qpu = QPU(num_qubits=3)
reg = Qubits(3, "reg", qpu)
reg.had()

refl = Reflect()

# Multiply |111⟩ by -1
refl.compute(reg)

# Multiply |000⟩ by -1
refl.compute(~reg)

# Multiply |100⟩ by i
refl.compute(reg[0] | ~reg[1:], theta=90)

# Multiply |110⟩ by e^(iπ/4)
refl.compute(reg==3, theta=(1, 4))

qpu.print_state_vector()

qpu.draw(show_qubricks=True)

In [ ]:
from psiqworkbench import QPU, Qubits
from psiqworkbench.qubricks import Reflect

qpu = QPU(num_qubits=3)
reg = Qubits(3, "reg", qpu)
reg.had()
qpu.print_state_vector()
refl = Reflect()

# Multiply |111⟩ by -1
refl.compute(~reg)
qpu.print_state_vector()